# Clean version of EEGnet

We will try to provide a clean version of the EEGnet network cited from this paper [insert link]

In [23]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, Dense
from keras.layers import BatchNormalization, Dropout
from keras.layers import MaxPool2D, Flatten
from keras.layers import Permute, ZeroPadding2D
from keras.activations import softmax,sigmoid
from keras.optimizers import Adam
from keras.losses import binary_crossentropy

np.random.seed(42)

# This function will allow us to build a model based on the EEGnet architecture

def EEGnet(ch=64, ts=159):
    # Start model
    model = Sequential()
    
    # Layer 1
    model.add(Conv2D(input_shape=(1,ch,ts), filters=16, kernel_size=(ch,1), data_format='channels_first'))
    model.add(BatchNormalization(axis=1))
    model.add(Permute((2,1,3)))
    model.add(Dropout(.25))
    
    # Padding to get N % 4
    model.add(ZeroPadding2D(padding=((0,1), (16,17)), data_format='channels_first'))
    
    # Layer 2
    
    model.add(Conv2D(filters=4, kernel_size=(2,32), activation='relu', data_format='channels_first'))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPool2D(pool_size=(2,4), data_format='channels_first'))
    model.add(Dropout(.25))
    
    # Padding to get N % 4
    model.add(ZeroPadding2D(padding=((4,3), (2,1)), data_format='channels_first'))
    
    # Layer 3 
    model.add(Conv2D(filters=4, kernel_size=(8,4), activation='relu', data_format='channels_first'))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPool2D(pool_size=(2,4), data_format='channels_first'))
    model.add(Dropout(.25))

    # Layer 4 
    model.add(Flatten())
    model.add(Dense(units=2, activation=softmax))
    #model.add(Dense(units=1, activation=sigmoid))

    model.compile(loss=binary_crossentropy,
              optimizer=Adam(), metrics=['accuracy'])
    
    return model

In [16]:
# TODO: Load the data from Feedback Error-Related Negativity (ERN)
# Training : Folder with .csv files
# Time | Channels (64) | EOG | FeedbackEvent
# Labels : single .csv file
# IdFeedBack | Prediction
# Test labels : https://www.kaggle.com/c/inria-bci-challenge/discussion/14515
import numpy as np
import glob

training_subjects = ['02', '06', '07', '11', '12', '13', '14', '16', '17', '18', '20', '21']
validation_subjects = ['22', '23', '24', '26']

file_names_training = []
file_names_val = []

for S_ in training_subjects:
    file_names_training = file_names_training + glob.glob('train/Data_S{}*.csv'.format(S_))
for S_ in validation_subjects:
    file_names_val = file_names_val + glob.glob('train/Data_S{}*'.format(S_))
    
# We can now get the file names, we will continue to experiment using only one .csv file
# The goal is to be able to build a feature array and labels array that we can feed into our EEGnet

['train/Data_S22_Sess01.csv',
 'train/Data_S22_Sess03.csv',
 'train/Data_S22_Sess02.csv',
 'train/Data_S22_Sess05.csv',
 'train/Data_S22_Sess04.csv',
 'train/Data_S23_Sess05.csv',
 'train/Data_S23_Sess04.csv',
 'train/Data_S23_Sess01.csv',
 'train/Data_S23_Sess03.csv',
 'train/Data_S23_Sess02.csv',
 'train/Data_S24_Sess02.csv',
 'train/Data_S24_Sess03.csv',
 'train/Data_S24_Sess01.csv',
 'train/Data_S24_Sess04.csv',
 'train/Data_S24_Sess05.csv',
 'train/Data_S26_Sess03.csv',
 'train/Data_S26_Sess02.csv',
 'train/Data_S26_Sess01.csv',
 'train/Data_S26_Sess05.csv',
 'train/Data_S26_Sess04.csv']

In [19]:
#test = np.loadtxt('train/Data_S02_Sess01.csv', delimiter=',', skiprows=1)
#test.shape